In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import pandas_udf, split, substring_index, element_at
from pyspark.sql.types import *
import pyspark.sql

# create a spark session
spark = SparkSession.builder.getOrCreate()

2021-12-14 01:35:37 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# load in data
df = spark.read.schema(
    StructType(
        [
            StructField('documents', StringType())
        ]
    )
 )\
.text("project2_data.txt")
# df.show()

In [8]:
df1 = df.withColumn('documentID', split(df['documents'], ' ').getItem(0)) \
       .withColumn('document', element_at(split("documents","^[^\s]*\s"),-1).cast("string"))
# df1.show(truncate=True)

In [10]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("document")
tokenized_df = tokenizer.transform(df1)

 #tokenized_df.show()

In [14]:
hashingTF = HashingTF(inputCol="words", outputCol="tf")
hashingTF.setNumFeatures(100000)
tf = hashingTF.transform(tokenized_df)
tf.show()

+--------------------+----------+--------------------+--------------------+--------------------+
|           documents|documentID|            document|               words|                  tf|
+--------------------+----------+--------------------+--------------------+--------------------+
|100152 purine nuc...|    100152|purine nucleoside...|[purine, nucleosi...|(100000,[618,1583...|
|10021296 protein ...|  10021296|protein kinase c ...|[protein, kinase,...|(100000,[92,615,4...|
|10021337 drosophi...|  10021337|drosophila kismet...|[drosophila, kism...|(100000,[92,539,1...|
|10021345 dis_card...|  10021345|dis_cardiac_dis h...|[dis_cardiac_dis,...|(100000,[671,1525...|
|10021362 sonic he...|  10021362|sonic hedgehog si...|[sonic, hedgehog,...|(100000,[92,539,1...|
|10021364 drosophi...|  10021364|drosophila tnf-re...|[drosophila, tnf-...|(100000,[92,1028,...|
|10021369 identifi...|  10021369|identification ap...|[identification, ...|(100000,[92,539,6...|
|10021376 intracel...|  100213

In [15]:
idf = IDF(minDocFreq=1)
idf.setInputCol("tf")
idf.setOutputCol("idf")
model = idf.fit(tf)

In [16]:
tfidf = model.transform(tf)

tfidf.show()

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|           documents|documentID|            document|               words|                  tf|                 idf|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|100152 purine nuc...|    100152|purine nucleoside...|[purine, nucleosi...|(100000,[618,1583...|(100000,[618,1583...|
|10021296 protein ...|  10021296|protein kinase c ...|[protein, kinase,...|(100000,[92,615,4...|(100000,[92,615,4...|
|10021337 drosophi...|  10021337|drosophila kismet...|[drosophila, kism...|(100000,[92,539,1...|(100000,[92,539,1...|
|10021345 dis_card...|  10021345|dis_cardiac_dis h...|[dis_cardiac_dis,...|(100000,[671,1525...|(100000,[671,1525...|
|10021362 sonic he...|  10021362|sonic hedgehog si...|[sonic, hedgehog,...|(100000,[92,539,1...|(100000,[92,539,1...|
|10021364 drosophi...|  10021364|drosophila tnf-re...|[d

In [18]:
res = tfidf.rdd.map(lambda x : (x.documentID,x.words,x.tf,x.idf,(None if x.idf is None else x.idf.values.sum())))

for r in res.take(10):
    print(r)


('100152', ['purine', 'nucleoside', 'gene_nucleoside_phosphorylase_gene', 'dis_chronic_lymphocytic_leukemia_dis', 'dis_cll_dis', '', 'purine', 'nucleoside', 'gene_nucleoside_phosphorylase_gene', 'gene_pnp_gene', 'enzyme', 'schematic', 'next', 'adenosine', 'deaminase', 'purine', 'salvage', 'pathway', 'demonstrate', 'cytochemical', 'peripheral', 'blood', 'lymphocyte', 'healthy', 'subject', 'dis_chronic_lymphocytic_leukemia_dis', 'dis_cll_dis', 'patient', '', 'enzyme', 'activity', 'confine', 'cytosol', '', 'healthy', 'subject', 'majority', 'lymphocyte', 'strong', 'reactive', 'gene_pnp_gene', 'whereas', 'rest', 'devoid', 'cytochemical', 'demonstrable', 'activity', '', 'percentage', 'pnp-positive', 'cell', 'large', 'correspond', 'number', 'e', 'rosette-forming', 'cell', 'inverse', 'proportional', 'number', 'ig-bearing', 'cell', '', 'six', 'seven', 'dis_cll_dis', 'patient', 'study', 'minor', 'percentage', 'lymphocyte', 'show', 'strong', 'gene_pnp_gene', 'activity', 'whereas', 'large', 'major